<a href="https://colab.research.google.com/github/sibat119/papers-review-code-impl/blob/main/lda/lda_with_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importing the Required Libraries**


In [2]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 31.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 11.3 MB 65.7 MB/s 
     |████████████████████████████████| 15.7 MB 180 kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=1060b2c070ef8880d32db219700dfcff1547dfca4d91f73c9303221db70ceecb
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take in

In [20]:
!pip install nltk
import nltk
nltk.download("stopwords")
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [63]:
import numpy as np
import pandas as pd
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy

#vis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


## **Load Data**

In [23]:
data = pd.read_csv("/content/Review.csv")
data.head()

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Num_words_text
0,0,226241,B002T0NWKE,A1G25CG7UWQ3XO,&#10003;Onoñda&#660;gegá&#660;,1,5,1,1309305600,Misleading discription-bad taste,they advertise that they contain aspartame die...,43
1,1,194272,B0009Y8AGI,A34Y9SS1F0H485,Bluemoon,0,6,1,1263600000,Yuck!,taste disgusting having struggle just keep dow...,27
2,2,359016,B003CIBPN8,ANDTRFIVWED40,R. Hauff,6,8,1,1275782400,"If you liked it in tetrapak box, don't buy now",bought this zico water hoping have good taste ...,53
3,3,402750,B000EQX6HO,A2W95LYEAIRDAC,"I. B. Lehman ""IBL""",1,1,1,1316044800,Great product...just not in small packages!,love terra chipswhen they bags from storei ord...,49
4,4,429211,B0006VM0LU,A32T7H92YWC0A2,Molly F,0,0,1,1317340800,Unfortunately unsalvageable,wanted desperately like this sauce smells tast...,35


In [24]:
stopwords = stopwords.words("english")

In [25]:
print(len(stopwords), stopwords)

179 ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

In [26]:
reviews = pd.DataFrame(data, columns= ['Text'])
print (reviews)

                                                   Text
0     they advertise that they contain aspartame die...
1     taste disgusting having struggle just keep dow...
2     bought this zico water hoping have good taste ...
3     love terra chipswhen they bags from storei ord...
4     wanted desperately like this sauce smells tast...
...                                                 ...
9995  rescued cats female urinary infectionsshe very...
9996  love zipfizz quick energy like shakey caffeine...
9997  much except that this tastes same campbells to...
9998  after having craving mint several days went lo...
9999  office hooked these ordered assorted case from...

[10000 rows x 1 columns]


In [31]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

# df = pd.DataFrame(['this was cheesy', 'she likes these books', 'wow this is great'], columns=['text'])
reviews['text_lemmatized'] = reviews.Text.apply(lemmatize_text)
print(reviews)

                                                   Text                                    text_lemmatized
0     they advertise that they contain aspartame die...  [they, advertise, that, they, contain, asparta...
1     taste disgusting having struggle just keep dow...  [taste, disgusting, having, struggle, just, ke...
2     bought this zico water hoping have good taste ...  [bought, this, zico, water, hoping, have, good...
3     love terra chipswhen they bags from storei ord...  [love, terra, chipswhen, they, bag, from, stor...
4     wanted desperately like this sauce smells tast...  [wanted, desperately, like, this, sauce, smell...
...                                                 ...                                                ...
9995  rescued cats female urinary infectionsshe very...  [rescued, cat, female, urinary, infectionsshe,...
9996  love zipfizz quick energy like shakey caffeine...  [love, zipfizz, quick, energy, like, shakey, c...
9997  much except that this tastes sa

In [55]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        if len(new) > 0:
          final.append(new[0])
    return(final)

data_words = gen_words(reviews)
reviews['words'] = reviews.text_lemmatized.apply(gen_words)

In [56]:
print(reviews['words'][0][:10])

['they', 'advertise', 'that', 'they', 'contain', 'aspartame', 'diet', 'flavor', 'boldly', 'state']


In [59]:
id2word = corpora.Dictionary(reviews['words'])

corpus = []
for words in reviews['words']:
  new = id2word.doc2bow(words)
  corpus.append(new)


In [61]:
print(corpus[0][:20])
word = id2word[[0][:1][0]]
print(word)

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 3), (11, 1), (12, 2), (13, 1), (14, 3), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2)]
advertise


In [ ]:

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

## **Vizualizing the Data**

In [64]:
pyLDAvis.enable_notebook()

In [67]:
# lda_viz = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
lda_viz = gensimvis.prepare(lda_model, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [68]:
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
26     0.379723  0.156266       1        1  22.129918
24     0.360644  0.113510       2        1  14.480546
20     0.127049 -0.230967       3        1   7.164689
6      0.103510 -0.321250       4        1   6.095349
16     0.016795 -0.057819       5        1   3.405104
15     0.020147 -0.064826       6        1   3.124618
13     0.014482 -0.052405       7        1   2.947861
17    -0.003571 -0.023687       8        1   2.843716
29    -0.014681 -0.008075       9        1   2.788105
7     -0.007362 -0.052055      10        1   2.664012
5     -0.020016 -0.002312      11        1   2.423827
12    -0.010249 -0.022540      12        1   2.314383
8     -0.006920 -0.016132      13        1   2.276139
18    -0.027683  0.006766      14        1   2.207836
2     -0.037823  0.019914      15        1   1.912320
19    -0.043249  0.023238      16        1   1.766355
0     -0.035513  0.015263      17        1   1.703325
23    -0.047568  0.028482      18        1   1.581127
10    -0.064799  0.039700      19        1   1.563456
3     -0.058874  0.035874      20        1   1.520674
1     -0.062321  0.038298      21        1   1.515129
22    -0.039403  0.033604      22        1   1.425260
11    -0.068780  0.042242      23        1   1.345694
14    -0.054056  0.033392      24        1   1.337450
27    -0.062786  0.038662      25        1   1.330062
21    -0.073001  0.045743      26        1   1.299869
9     -0.062477  0.040393      27        1   1.285336
28    -0.071967  0.046862      28        1   1.280620
4     -0.074011  0.045708      29        1   1.174955
25    -0.075241  0.048151      30        1   1.092264, topic_info=           Term          Freq         Total Category  logprob  loglift
82         this  12236.000000  12236.000000  Default  30.0000  30.0000
28         they   4909.000000   4909.000000  Default  29.0000  29.0000
19         like   4973.000000   4973.000000  Default  28.0000  28.0000
116       these   4527.000000   4527.000000  Default  27.0000  27.0000
14       flavor   3180.000000   3180.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
3090       foil     39.857113     40.800680  Topic30  -4.8081   4.4935
2578       busy     39.766834     40.710402  Topic30  -4.8104   4.4935
2121       safe     39.491243     40.434810  Topic30  -4.8173   4.4933
11069    bakery     38.711991     39.655558  Topic30  -4.8372   4.4928
15345  decadent     34.841930     35.785497  Topic30  -4.9426   4.4902

[959 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
266      20  0.996687        able
247       1  0.999312       about
2493     29  0.979039    absolute
818      17  0.997164  absolutely
1659      5  0.986800      across
...     ...       ...         ...
420       2  0.999508        your
860      11  0.995449       youre
403      11  0.995953    yourself
2990     22  0.992387       youve
1471      5  0.994805       yummy

[953 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[27, 25, 21, 7, 17, 16, 14, 18, 30, 8, 6, 13, 9, 19, 3, 20, 1, 24, 11, 4, 2, 23, 12, 15, 28, 22, 10, 29, 5, 26])